In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install -r https://raw.githubusercontent.com/abbbe/eye-on-stick/main/requirements.txt
    !git clone https://github.com/abbbe/eye-on-stick
    %cd eye-on-stick

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import os, urllib
import mlflow, git

from stable_baselines.common.cmd_util import make_vec_env
from stable_baselines.common.vec_env import VecNormalize

import matplotlib.pyplot as plt
%matplotlib inline

from lib import eos
from lib.eos import EyeOnStickEnv

In [ ]:
from PIL import Image, ImageDraw
from IPython import display
from io import BytesIO

def showarray(img_array):
    buf = BytesIO()
    Image.fromarray(np.uint8(img_array)).save(buf, 'png')
    display.display(display.Image(data=buf.getvalue()))

In [ ]:
#assert not git.Repo().is_dirty()

In [ ]:
from stable_baselines import PPO2

def run(N_JOINTS, N_SEGS=1, POLICY_CLASS=PPO2, MODEL_NAME='MlpLnLstmPolicy'):
    env = make_vec_env(lambda: EyeOnStickEnv(N_JOINTS, N_SEGS), n_envs=eos.N_ENVS)
    env = VecNormalize(env)
    
    run_name = f'{N_JOINTS}J{N_SEGS}S'

    with mlflow.start_run(run_name=run_name) as parent_run:
        mlflow_artifacts_dir = urllib.request.url2pathname(urllib.parse.urlparse(mlflow.get_artifact_uri()).path)

        mlflow.log_param("policy_class", POLICY_CLASS.__name__)
        mlflow.log_param("model_name", MODEL_NAME)

        tensorboard_logdir = os.path.join(mlflow_artifacts_dir, "tensorboard_log")
        os.makedirs(tensorboard_logdir, exist_ok=False)

        model = POLICY_CLASS(MODEL_NAME, env, verbose=1, tensorboard_log=tensorboard_logdir)

        for era in range(eos.N_ERAS):
            with mlflow.start_run(run_name=f'era={era} sqrt', nested=True) as child_run:
                all_alphas, all_rewards = [], []

                obs = env.env_method('reset', reset_pose=True)

                for _ in range(eos.N_STEPS):
                    display.clear_output(wait=True)
                    showarray(env.render(mode='rgb_array'))

                    actions, _ = model.predict(obs, deterministic=True)
                    obs, rewards, _dones, infos = env.step(actions)

                    all_alphas.append([info['alpha'] for info in infos])                
                    all_rewards.append(rewards)

                mlflow.log_metric(key="alpha_mean", value=np.mean(all_alphas), step=era)
                mlflow.log_metric(key="alpha_std", value=np.std(all_alphas), step=era)
                mlflow.log_metric(key="reward_total", value=np.sum(all_rewards), step=era)
                mlflow.log_metric(key="reward_mean", value=np.mean(all_rewards), step=era)
                mlflow.log_metric(key="reward_std", value=np.std(all_rewards), step=era)

                model.learn(eos.N_LEARN_EPOCHS)
                #mlflow.keras.save_model(model, f"era{era}")

        env.close()

In [ ]:
for nj in [1, 2]:
    for _ in range(10):
        run(N_JOINTS=nj)